# Implementation of loading data and doing some analysis of it.

In [9]:
import random
import math 
import numpy as np
import time
import csv
import sys


def readAbaloneCsv(): # This function is written for leaf.csv, because of order of features and labels
    csvOpen = open('abalone.csv')
    csvReader = csv.reader(csvOpen) # using csv library to read iris.csv
    dataset = []
    for row in csvReader:
        dataset.append(row) # to csvreader to list
    random.shuffle(dataset) # shuffling list for k-fold-validation
    dataset_features = []
    dataset_labels = []
    for data in dataset: # iterating over dataset
        label = data.pop(len(data)-1) ## We are droping class and save it to label
        dataset_labels.append(label) # saving label to labels
        dataset_features.append(data) # saving features but data is full of string, change to float list
    return dataset_labels,dataset_features # returning 

def findLabels(labels): # This function finds how many different labels is there. for confusion matrix
    classes = []
    for label in labels:
        try:
            classes.index(label)
        except:
            classes.append(label)
    return classes

def countClasses(labels): # This function creates a dictionary of classes and number of them
    counts = {}
    class_names = findLabels(labels)
    for label in labels:
        if label not in counts:
            counts[label] = 0
        counts[label] +=1
    return counts

def isNumeric(s): # This function finds if the given string is numeric or not
    try:
        float(s)
        return True
    except ValueError:
        return False

def findAttributeTypes(features): # This function finds attribute_types of given vector.
    attribute_types = []
    for col in features:
        if(isNumeric(col)):
            attribute_types.append(1)
        else:
            attribute_types.append(0)
    return attribute_types

def makeNumeric(features,attribute_types): # This function casts all of numeric features.
    for col in range(0,len(attribute_types)): # Iterate attribute vector
        if attribute_types[col] == 1: # If given attribute is numeric
            for row in features:      # Make all this col numeric
                row[col] = float(row[col])
    return features # Return features



    


labels,features = readAbaloneCsv() # read abolone.csv
feature_names = ["Sex","Lenght","Diameter","Height","Whole weight",
                 "Shucked weight","Viscera weight","Shell weight","Rings"]
print("There is ",len(labels)," data in our dataset.")
count_classes = countClasses(labels)
print("There is total ",len(count_classes),"different classes and numbers of them")
classes = findLabels(labels)
for key in count_classes:
    print(key,":",count_classes[key])
print("Feature names are:")
print(feature_names)
print("Example features are:")
attribute_types = findAttributeTypes(features[0])
print(features[0])
print("Our attribute vector will be:")
print(attribute_types)
features = makeNumeric(features,attribute_types)
print("If we arrange our data with that vector, our data will be:")
print(features[0])



There is  4177  data in our dataset.
There is total  28 different classes and numbers of them
9 : 689
6 : 259
12 : 267
16 : 67
11 : 487
10 : 634
17 : 58
15 : 103
8 : 568
14 : 126
7 : 391
13 : 203
5 : 115
20 : 26
4 : 57
3 : 15
21 : 14
1 : 1
18 : 42
19 : 32
2 : 1
26 : 1
23 : 9
27 : 2
24 : 2
22 : 6
25 : 1
29 : 1
Feature names are:
['Sex', 'Lenght', 'Diameter', 'Height', 'Whole weight', 'Shucked weight', 'Viscera weight', 'Shell weight', 'Rings']
Example features are:
['F', '0.62', '0.51', '0.175', '1.2705', '0.5415', '0.323', '0.3225']
Our attribute vector will be:
[0, 1, 1, 1, 1, 1, 1, 1]
If we arrange our data with that vector, our data will be:
['F', 0.62, 0.51, 0.175, 1.2705, 0.5415, 0.323, 0.3225]


# Implementation of Decision Tree

In [14]:
class Leaf:
    # End condition for our tree.
    # It contains count of class which has impurity equals to 0
    def __init__(self,labels):
        self.predictions = countClasses(labels)
        
class Node:
    # Decision node for our tree.
    # It contains seperator which we use to seperate our data 
    # And also it contains a true branch if our decision is correct
    #                      a false branch if our decision is incorrect
    def __init__(self,
                 seperator,
                 true_branch,
                 false_branch):
        self.seperator = seperator
        self.true_branch = true_branch
        self.false_branch = false_branch        

        
class Seperator:
    # This class makes the decision for seperation.
    # It contains column (column index of features array, ["Sex"=0,"Lenght"=1,"Diameter"=2,"Height"=3,"Whole weight"=4,
    #             "Shucked weight"=5,"Viscera weight"=6,"Shell weight"=7,"Rings"=8])
    #             value, value of given column
    #             attribute_types, attribute vector
    def __init__(self, given_feature, value,attribute_types):
        self.column = given_feature
        self.value = value
        self.attribute_types = attribute_types

    def decide(self, example):
        # Compare the feature value in an example to the
        # feature value in this Seperator.
        val = example[self.column]
        try:
            if self.attribute_types[self.column]==1: # If type is numeric
                return val >= self.value
            else: # If type is string
                return val == self.value
        except TypeError:
                return val == self.value
            
def partition(features,labels, seperator): # This function seperates the data, by given seperator
    true_features = []
    true_labels = []
    false_features= []
    false_labels = []
    for i in range(0,len(features)): # Iterate features and labels
        if seperator.decide(features[i]): # If this feature is true
            true_features.append(features[i]) # Store them to true ones
            true_labels.append(labels[i])
        else: # else
            false_features.append(features[i]) # Store them to false ones
            false_labels.append(labels[i])
    return true_features,true_labels,false_features,false_labels # return them

def calculateImpurity(labels):
    counts = countClasses(labels)
    purity = 0
    for label in counts:
        prob_of_label = counts[label] / float(len(labels))
        purity += prob_of_label**2
    return 1-purity

def gain( true_labels,false_labels, current_uncertainty):
    p = float(len(true_labels)) / (len(true_labels) + len(false_labels))
    return current_uncertainty - p * calculateImpurity(true_labels) - (1 - p) * calculateImpurity(false_labels)

def find_best_split(train_features,train_labels,attribute_types):
    best_gain = 0  # keep track of the best information gain
    best_seperator = None  # keep train of the feature / value that produced it
    current_uncertainty = calculateImpurity(train_labels)
    values = []
    for col in range(0,len(train_features[0])): # for each feature
        #values = set([train_features[col] for row in train_features])  # unique values in the column
        values = []
        for row in train_features:
            try:
                values.index(row[col])
            except:
                values.append(row[col])
        for val in values:  # for each value
            
            seperator = Seperator(col, val,attribute_types)

            # try splitting the dataset
            true_features,true_labels,false_features,false_labels = partition(train_features,train_labels, seperator)

            # Skip this split if it doesn't divide the
            # dataset.
            if len(true_features) == 0 or len(false_features) == 0:
                continue

            # Calculate the information gain from this split
            gain = gain( true_labels,false_labels, current_uncertainty)

            # You actually can use '>' instead of '>=' here
            # but I wanted the tree to look a certain way for our
            # toy dataset.
            if gain >= best_gain:
                best_gain, best_seperator = gain, seperator

    return best_gain, best_seperator

def build_dt(X, y, attribute_types, options = 0):
    # Try partitioing the dataset on each of the unique attribute,
    # calculate the information gain,
    # and return the seperator that produces the highest gain.
    #print(X,y)
    gain, seperator = find_best_split(X,y,attribute_types)
 
    # Base case: no further info gain
    # Since we can ask no further seperators,
    # we'll return a leaf.
    if gain == 0:
        return Leaf(y)

    # If we reach here, we have found a useful feature / value
    # to partition on.
    true_features,true_labels,false_features,false_labels = partition(X,y, seperator)

    # Recursively build the true branch.
    true_branch = build_dt(true_features,true_labels,attribute_types)

    # Recursively build the false branch.
    false_branch = build_dt(false_features,false_labels,attribute_types)

    # Return a seperator node.
    # This records the best feature / value to ask at this point,
    # as well as the branches to follow
    # dependingo on the answer.
    return Node(seperator, true_branch, false_branch)

# Implementation of prediction to decision tree

In [15]:
def predict_dt(dt, X, options = 0):
    predictions = []
    for i in range(0,len(X)):
        predictions.append(predict(dt,X[i],options))
    return predictions

def predict(dt, feature, options=0):
    """See the 'rules of recursion' above."""
    # Base case: we've reached a leaf
    if isinstance(dt, Leaf):
        return next(iter(dt.predictions))
        
    # Decide whether to follow the true-branch or the false-branch.
    # Compare the feature / value stored in the node,
    # to the example we're considering.
    # print(dt.seperator,X[dt.seperator.column])
    if dt.seperator.decide(feature):
        return predict(dt.true_branch,feature,options)
    else:
        return predict(dt.false_branch,feature,options)

# Implementation of k-fold validation

In [16]:
def split(labels,features,n,k): # this function splits dataset using k-fold-cross-validation
    train_labels = []
    train_features = []
    test_labels = []
    test_features = []

    for i in range(0,len(labels)): # iterating dataset
        # This condition works like this : goo.gl/images/WNkSSV n is our iteration number k is our splitting factor.
        if i >= (len(labels)/k)*(n-1) and i < (len(labels)/k)*(n-1)+len(labels)/k: #splitting test data with n
            test_labels.append(labels[i])                                          #nth test data is chosen
            test_features.append(features[i])
        else:
            train_labels.append(labels[i])
            train_features.append(features[i])

    return train_labels,train_features,test_labels,test_features # returning splitted datas


def kfold(k,labels,features,classes,attribute_types):
    accuracies = []
    result = []
    print("Dataset will be splitted to ",k," equal pieces")
    for i in range(1,k+1): # do k-fold-validation
        train_labels,train_features,test_labels,test_features = split(labels,features,i,k)
        print(i,"th/",k," will be used as test data")
        print("Train dataset size is",len(train_labels),"Test dataset size is",len(test_labels))
        decision_tree = build_dt(train_features, train_labels, attribute_types)
        #print("Decision tree bitti",len(test_features),len(test_features[0]))
        predictions = predict_dt(decision_tree,test_features)
        count=0
        for i in range(0,len(predictions)):
            if predictions[i]==test_labels[i]:
                count+=1
        result = count/len(predictions)
        print("This accuracy is ",result)
        accuracies.append(result) # find accuracies
    results = 0
    for i in range(0,len(accuracies)): # iterating accuracies
        results += accuracies[i] # add accuracies
    accuracy = results/len(accuracies) # calculate average

    print("accuracies:",accuracies)
    print("Average accuracy:",accuracy,"Total samples:",len(labels))

# Doing 5-fold validation

In [17]:
print("k-fold validation for 5")    
kfold(5,labels,features,classes,attribute_types)

k-fold validation for 5
Dataset will be splitted to  5  equal pieces
1 th/ 5  will be used as test data
Train dataset size is 3341 Test dataset size is 836
This accuracy is  0.1854066985645933
2 th/ 5  will be used as test data
Train dataset size is 3342 Test dataset size is 835
This accuracy is  0.19041916167664671
3 th/ 5  will be used as test data
Train dataset size is 3341 Test dataset size is 836
This accuracy is  0.2021531100478469
4 th/ 5  will be used as test data
Train dataset size is 3342 Test dataset size is 835
This accuracy is  0.19520958083832335
5 th/ 5  will be used as test data
Train dataset size is 3342 Test dataset size is 835
This accuracy is  0.18682634730538922
accuracies: [0.1854066985645933, 0.19041916167664671, 0.2021531100478469, 0.19520958083832335, 0.18682634730538922]
Average accuracy: 0.19200297968655988 Total samples: 4177


Accuracy is pretty low. I think reason for this, we have too many classes but that classes are not represented well.
For example class 1 only has 1 data. So we must train tree with more data. For example we will try 10/11 of dataset to train tree.

# Doing 11-fold validation

In [63]:
print("k-fold validation for 11")    
kfold(11,labels,features,classes,attribute_types)

k-fold validation for 11
Dataset will be splitted to  11  equal pieces
1 / 11  will be used as test data
Train dataset size is 3797 Test dataset size is 380
This accuracy is  0.2026315789473684
2 / 11  will be used as test data
Train dataset size is 3797 Test dataset size is 380
This accuracy is  0.18157894736842106
3 / 11  will be used as test data
Train dataset size is 3797 Test dataset size is 380
This accuracy is  0.2
4 / 11  will be used as test data
Train dataset size is 3798 Test dataset size is 379
This accuracy is  0.1820580474934037
5 / 11  will be used as test data
Train dataset size is 3797 Test dataset size is 380
This accuracy is  0.17894736842105263
6 / 11  will be used as test data
Train dataset size is 3797 Test dataset size is 380
This accuracy is  0.22105263157894736
7 / 11  will be used as test data
Train dataset size is 3797 Test dataset size is 380
This accuracy is  0.21842105263157896
8 / 11  will be used as test data
Train dataset size is 3798 Test dataset size 

In [18]:
print("k-fold validation for 3")    
kfold(3,labels,features,classes,attribute_types)

k-fold validation for 3
Dataset will be splitted to  3  equal pieces
1 th/ 3  will be used as test data
Train dataset size is 2784 Test dataset size is 1393
This accuracy is  0.17875089734386218
2 th/ 3  will be used as test data
Train dataset size is 2785 Test dataset size is 1392
This accuracy is  0.1997126436781609
3 th/ 3  will be used as test data
Train dataset size is 2785 Test dataset size is 1392
This accuracy is  0.2025862068965517
accuracies: [0.17875089734386218, 0.1997126436781609, 0.2025862068965517]
Average accuracy: 0.1936832493061916 Total samples: 4177


As we can see there is a little improvement in accuracy. But still results are bad.